In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf 
print([tf.__version__, tf.test.is_gpu_available()])

from sklearn.model_selection import KFold
from tensorboard.plugins.hparams import api as hp

['2.0.0', False]

In [2]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64,128,256,512]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1,0.2,0.3,0.4,0.5]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [3]:
def train_model(hparams, tune=False):
  input_shape = (28,28,1)
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape, padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  
  n_split = 4
  ep = 1 if (tune) else 3
  
  for train_index,test_index in KFold(n_split).split(x_data):
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    x_train,x_test=x_data[train_index],x_data[test_index]
    y_train,y_test=y_data[train_index],y_data[test_index]
  
    model.fit(x_train, y_train,epochs=ep, verbose=2)
  
    _, accuracy = model.evaluate(x_test,y_test, verbose=2)
    if (tune): break
    
  #accuracy = np.array(accuracy_list).max() 
  return accuracy, model

In [4]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy, _ = train_model(hparams, tune=True)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
    return accuracy

In [5]:
# File location and type
file_location = "/FileStore/tables/train.csv"
test_location = "/FileStore/tables/test.csv"
file_type = "csv"

#customSchema = StructType()

# The applied options are for CSV files. For other file types, these will be ignored.
dataDF = sqlContext.read.format(file_type).options(delimiter=',', header='true', inferschema='true').load(file_location)
testDF = sqlContext.read.format(file_type).options(delimiter=',', header='true', inferschema='true').load(test_location)

x_data = dataDF.drop('label').toPandas()
y_data = dataDF.select('label').toPandas().values

x_data = (x_data.values/255.0).reshape(-1,28,28,1)

#(validDF, trainDF) = dataDF.randomSplit([15.0, 85.0])

#x_train = trainDF.drop('label').toPandas()
#y_train = trainDF.select('label').toPandas()

#x_valid = validDF.drop('label').toPandas()
#y_valid = validDF.select('label').toPandas()

#x_train = x_train.values.reshape(-1,28,28,1)/255.0
#x_valid = x_valid.values.reshape(-1,28,28,1)/255.0

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [6]:
session_num = 0
result = dict()
for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.values):
    hparams = {
    HP_NUM_UNITS: num_units,
    HP_DROPOUT: dropout_rate
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    acc = run('logs/hparam_tuning/' + run_name, hparams)
    result.update({(num_units, dropout_rate): acc})
    session_num += 1

--- Starting trial: run-0
{'num_units': 64, 'dropout': 0.1}
Model: "sequential_24"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
conv2d_48 (Conv2D) (None, 28, 28, 32) 832 
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 14, 14, 32) 0 
_________________________________________________________________
conv2d_49 (Conv2D) (None, 14, 14, 64) 51264 
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 7, 7, 64) 0 
_________________________________________________________________
flatten_24 (Flatten) (None, 3136) 0 
_________________________________________________________________
dense_48 (Dense) (None, 64) 200768 
_________________________________________________________________
dropout_24 (Dropout) (None, 64) 0 
_________________________________________________________________
dense_49 (Dense) (None, 10) 650 
=================================================================
Total params: 253,514
Trainable params: 253,514
Non-trainable params: 0
_________________________________________________________________
None
Train on 31500 samples
31500/31500 - 41s - loss: 0.1878 - accuracy: 0.9401
10500/1 - 5s - loss: 0.0502 - accuracy: 0.9813
--- Starting trial: run-1
{'num_units': 64, 'dropout': 0.2}
Model: "sequential_25"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
conv2d_50 (Conv2D) (None, 28, 28, 32) 832 
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 14, 14, 32) 0 
_________________________________________________________________
conv2d_51 (Conv2D) (None, 14, 14, 64) 51264 
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 7, 7, 64) 0 
_________________________________________________________________
flatten_25 (Flatten) (None, 3136) 0 
_________________________________________________________________
dense_50 (Dense) (None, 64) 200768 
_________________________________________________________________
dropout_25 (Dropout) (None, 64) 0 
_________________________________________________________________
dense_51 (Dense) (None, 10) 650 
=================================================================
Total params: 253,514
Trainable params: 253,514
Non-trainable params: 0
_________________________________________________________________
None
Train on 31500 samples
31500/31500 - 39s - loss: 0.2320 - accuracy: 0.9283
10500/1 - 4s - loss: 0.0328 - accuracy: 0.9818
--- Starting trial: run-2
{'num_units': 64, 'dropout': 0.3}
Model: "sequential_26"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
conv2d_52 (Conv2D) (None, 28, 28, 32) 832 
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 14, 14, 32) 0 
_________________________________________________________________
conv2d_53 (Conv2D) (None, 14, 14, 64) 51264 
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 7, 7, 64) 0 
_________________________________________________________________
flatten_26 (Flatten) (None, 3136) 0 
_________________________________________________________________
dense_52 (Dense) (None, 64) 200768 
_________________________________________________________________
dropout_26 (Dropout) (None, 64) 0 
_________________________________________________________________
dense_53 (Dense) (None, 10) 650 
=================================================================
Total params: 253,514
Trainable params: 253,514
Non-trainable params: 0
_________________________________________________________________
None
Train 

In [7]:
param = max(result, key=result.get)
print(param, result[param])

(512, 0.3) 0.98438096

In [8]:
hparam = {HP_NUM_UNITS: param[0], HP_DROPOUT: param[1]}
accuracy, model = train_model(hparam)

Train on 31500 samples
Epoch 1/3
31500/31500 - 41s - loss: 0.1561 - accuracy: 0.9521
Epoch 2/3
31500/31500 - 41s - loss: 0.0500 - accuracy: 0.9846
Epoch 3/3
31500/31500 - 41s - loss: 0.0354 - accuracy: 0.9890
10500/1 - 5s - loss: 0.0175 - accuracy: 0.9897
Train on 31500 samples
Epoch 1/3
31500/31500 - 42s - loss: 0.0347 - accuracy: 0.9887
Epoch 2/3
31500/31500 - 41s - loss: 0.0231 - accuracy: 0.9931
Epoch 3/3
31500/31500 - 41s - loss: 0.0169 - accuracy: 0.9943
10500/1 - 4s - loss: 0.0134 - accuracy: 0.9929
Train on 31500 samples
Epoch 1/3
31500/31500 - 41s - loss: 0.0224 - accuracy: 0.9935
Epoch 2/3
31500/31500 - 42s - loss: 0.0136 - accuracy: 0.9962
Epoch 3/3
31500/31500 - 41s - loss: 0.0114 - accuracy: 0.9968
10500/1 - 4s - loss: 0.0077 - accuracy: 0.9954
Train on 31500 samples
Epoch 1/3
31500/31500 - 41s - loss: 0.0153 - accuracy: 0.9952
Epoch 2/3
31500/31500 - 41s - loss: 0.0110 - accuracy: 0.9967
Epoch 3/3
31500/31500 - 41s - loss: 0.0077 - accuracy: 0.9978
10500/1 - 4s - loss: 0.0028 - accuracy: 0.9981

In [9]:
prediction = model.predict(testDF.toPandas().astype('float32').values.reshape(-1,28,28,1)/255.0)

pre_lbl = np.argmax(prediction,axis=1)
output = pd.DataFrame(pre_lbl).reset_index()
output['index'] += 1
output = output.rename({'index': 'ImageId', 0: 'Label'}, axis = 1)
outputDF = spark.createDataFrame(output)

#save prediction as csv
(outputDF
 .coalesce(1)
 .write
 .format("csv")
 .options(delimiter=',', header="true")
 .save('FileStore/submission2'))

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "